In [3]:
import geopandas as gpd
import rasterio
import rasterio.features
from rasterio.features import rasterize
import numpy as np

def create_parcel_mask(parcel_file, satellite_file):
    # Read the parcel data using geopandas
    parcels = gpd.read_file(parcel_file)

    # Read the satellite image using rasterio
    with rasterio.open(satellite_file) as src:
        satellite_image = src.read()
        meta = src.meta.copy()

    # Create an empty mask with the same shape as the satellite image
    mask = np.zeros((meta['height'], meta['width']), dtype=np.uint8)

    # Make sure the parcel data is in the same CRS as the satellite image
    if parcels.crs != meta['crs']:
        parcels = parcels.to_crs(meta['crs'])

    # Rasterize the parcel geometries onto the mask
    mask = rasterize(
        shapes=parcels.geometry,
        out=mask,
        transform=meta['transform'],
        all_touched=True,
        fill=0,
        default_value=1
    )

    # Update the metadata for the mask
    meta.update({
        'count': 1,
        'dtype': 'uint8'
    })

    return mask, meta

# Specify the paths to the parcel and satellite image files
parcel_file = "/workspaces/Satelite/data/parcels/ZH_ZH_parcel_1.gpkg"
satellite_file = "/workspaces/Satelite/data/satellite/ZH_ZH_upscaled_parcel_1.tif"

# Create the parcel mask
parcel_mask, mask_meta = create_parcel_mask(parcel_file, satellite_file)

# Save the parcel mask as a new GeoTIFF file
mask_file = "/workspaces/Satelite/data/mask/ZH_ZH_parcel_1_mask.tif"
with rasterio.open(mask_file, 'w', **mask_meta) as dst:
    dst.write(parcel_mask, 1)